In [1]:
import numpy as np
import math
from matplotlib import pyplot as plt
from scipy.optimize import fsolve

In [2]:
'''
###############################################################################
# A class for wave properties
# Requirements:
# from scipy.optimize import fsolve
###############################################################################

'''

class RealWave:
    '''
    Class for calculating a set of physical properties and non-dimensional numbers
    of a monochromic wave given some quantities. All the quantities 
    are in SI units.
    '''
    
    def __init__(self, g = 9.8, sigma = 0.074, rho = 1000, rho_air = 1.225, 
                 mu = 8.9e-4, mu_air = 17.4e-6):
        '''
        Parameters
        ----------

        g : gravity acceleration (m/s^2)  
        sigma : surface tension of water/air interface (N/m)
        rho : density of water (kg/m^3)
        rho_air : density of air
        
        self.k : wave number (1/m)
        self.omega : wave frequency (1/s)
        self.c : phase speed (m/s)
        self.wl : wavelength (m)
        self.Bo : Bond number 
        
        '''
        self.g, self.sigma, self.rho, self.rho_air, self.mu, self.mu_air = \
        g, sigma, rho, rho_air, mu, mu_air
        self.k, self.omega, self.c, self.wl, self.Bo, self.Re_wave, self.Re_air = 0, 0, 0, 0, 0, 0, 0
        
    def k2omega(self,k):
        self.k = k
        # Gravity-capillary wave dispersion relation
        self.omega = (self.g*self.k + self.sigma*self.k**3/self.rho)**0.5
        self.c = self.omega/self.k
        self.wl = 2*np.pi/self.k
        self.Bo =  (self.rho-self.rho_air)*self.g/self.sigma/self.k**2
        print("Given k = %g (1/m), calculated omega = %g (1/s), period = %g (s), phase speed c = %g (m/s), wavelength = %g (m), Bo = %g" 
              %(self.k, self.omega, 2*np.pi/self.omega, self.c, self.wl, self.Bo))

    # Implicit function of w(k)
    def omega2k(self,omega):
        self.omega = omega
        k = fsolve(lambda k : (self.g*k + self.sigma*k**3/self.rho)**0.5 - omega, 0)
        self.k = k[0]
        self.c = self.omega/self.k
        self.wl = 2*np.pi/self.k
        self.Bo =  (self.rho-self.rho_air)*self.g/self.sigma/self.k**2
        print("Given omega = %g (1/s), calculated k = %g (1/m), phase speed c = %g (m/s), wavelength = %g (m), Bo = %g" 
              %(self.omega, self.k, self.c, self.wl, self.Bo))
              
    # If Bond number is given instead of k
    def Bo2k(self,Bo):
        self.Bo = Bo
        self.k = ((self.rho-self.rho_air)*self.g/Bo/self.sigma)**0.5
        self.wl = 2*np.pi/self.k
        self.omega = (self.g*self.k + self.sigma*self.k**3/self.rho)**0.5
        self.c = self.omega/self.k
        print("Given Bo = %g, calculated lambda = %g (m), k = %g (1/m), omega = %g (1/s), phase speed c = %g (m/s)" 
              %(self.Bo, self.wl, self.k, self.omega, self.c))
     
    def Re(self, UstarRatio = 0, L0 = 1.):
        self.Re_wave = self.rho*self.c*(2*np.pi/self.k)/self.mu
        self.Re_air = self.rho_air*self.c*UstarRatio*(L0/2.)/self.mu_air
        c_simu = (1/2/np.pi*(1+1/self.Bo))**0.5
        self.Re_nominal = self.Re_wave/c_simu
        print("Re_wave = %g, Re_air = %g, Re in older version = %g" %(self.Re_wave, self.Re_air, self.Re_nominal))

In [6]:
wave1 = RealWave(g = 9.8, sigma = 0.074, rho = 1000, rho_air = 1.225, mu = 8.9e-4, mu_air = 17.4e-6)
wave1.omega2k(2*2*np.pi)
wave1.omega2k(1.18*2*np.pi)

Given omega = 12.5664 (1/s), calculated k = 16.0822 (1/m), phase speed c = 0.781382 (m/s), wavelength = 0.390691 (m), Bo = 511.41
Given omega = 7.41416 (1/s), calculated k = 5.60783 (1/m), phase speed c = 1.32211 (m/s), wavelength = 1.12043 (m), Bo = 4206.04


In [7]:
wave1.k*0.2*10**(-3)

0.0011215652760303908

In [4]:
# Lin-log waves
wave1 = RealWave(g = 9.8, sigma = 0.074, rho = 1000, rho_air = 1.225, mu = 8.9e-4, mu_air = 17.4e-6)
wave1.Bo2k(1.47)

Given Bo = 1.47, calculated lambda = 0.0209463 (m), k = 299.966 (1/m), omega = 70.2637 (1/s), phase speed c = 0.234239 (m/s)


In [5]:
wave1 = RealWave(g = 9.8, sigma = 0.074, rho = 1000, rho_air = 1.225, mu = 8.9e-4, mu_air = 17.4e-6)
wave1.omega2k(2*np.pi/0.6)

Given omega = 10.472 (1/s), calculated k = 11.1795 (1/m), phase speed c = 0.936714 (m/s), wavelength = 0.562029 (m), Bo = 1058.32


In [104]:
0.19/0.27

0.7037037037037037

In [11]:
wave1 = RealWave(g = 9.8, sigma = 0.074, rho = 1000, rho_air = 1.225, mu = 8.9e-4, mu_air = 17.4e-6)
wave1.Bo2k(10000)
wave1.Re()

Given Bo = 10000, calculated lambda = 1.72762 (m), k = 3.6369 (1/m), omega = 5.97036 (1/s), phase speed c = 1.64161 (m/s)
Re_wave = 3.1866e+06, Re_air = 0, Re in older version = 7.98723e+06


In [83]:
# Linear waves
Bo = 1.47; k = 2*np.pi; Re = 10600
wave2 = RealWave(g = 1, sigma = 1/(Bo*k**2), rho = 1, rho_air = 1.225*10**(-3), mu = 1/Re, mu_air = 1/Re/8.9e-4*17.4e-6)
wave2.k2omega(2*np.pi)

Given k = 6.28319 (1/m), calculated omega = 3.24922 (1/s), period = 1.93375 (s), phase speed c = 0.51713 (m/s), wavelength = 1 (m), Bo = 1.4682


In [66]:
# linear Re20000 = lin-log Re8368
# linear Re10000 = lin-log Re4184

In [59]:
# Turbulence wave
Bo = 0.845; k_ = 4; Re = 10000; g_ = 1; rho1_ = 1
sigma_ = rho1_/(Bo*(k_)**2)
c_ = (g_/k_+sigma_/rho1_*k_)**0.5
mu1_ = (2.*np.pi/k_)*c_*rho1_/Re
print('sigma = %g' %sigma_)
print('nu1 = %g, nu2 = %g' %(mu1_/rho1_,mu1_/rho1_/8.9e-4*17.4e-6/1.225e-3))
wave2 = RealWave(g = g_, sigma = sigma_, rho = rho1_, rho_air = 1.225e-3*rho1_, mu = mu1_, mu_air = mu1_/8.9e-4*17.4e-6)
wave2.k2omega(4)
# RE of air based on wave length
wave2.Re(0.5, 2*np.pi/4)

sigma = 0.0739645
nu1 = 0.000116054, nu2 = 0.00185218
Given k = 4 (1/m), calculated omega = 2.95529 (1/s), period = 2.12608 (s), phase speed c = 0.738822 (m/s), wavelength = 1.5708 (m), Bo = 0.843965
Re_wave = 10000, Re_air = 156.645, Re in older version = 16958


In [100]:
# Turbulence wave real-world
k_ = 4; g_ = 9.8; rho1_ = 1000; mu = 8.9e-4
sigma_ = 0.074
c_ = (g_/k_+sigma_/rho1_*k_)**0.5
Re = (2.*np.pi/k_)*c_*rho1_/mu
print('sigma = %g' %sigma_)
print('nu1 = %g, nu2 = %g' %(mu1_/rho1_,mu1_/rho1_/8.9e-4*17.4e-6/1.225e-3))
wave2 = RealWave(g = g_, sigma = sigma_, rho = rho1_, rho_air = 1.225e-3*rho1_, mu = mu1_, mu_air = mu1_/8.9e-4*17.4e-6)
wave2.k2omega(4)
# RE of air based on wave length
wave2.Re(0.5, 2*np.pi/4)

sigma = 0.074
nu1 = 1.16054e-07, nu2 = 1.85218e-06
Given k = 4 (1/m), calculated omega = 6.26137 (1/s), period = 1.00348 (s), phase speed c = 1.56534 (m/s), wavelength = 1.5708 (m), Bo = 8266.89
Re_wave = 2.1187e+07, Re_air = 331884, Re in older version = 5.31047e+07
